# Convert JSON into CSV

Here's the definition of a function you can use to convert a Python value—originally parsed from a JSON string—into an equivalent CSV string.

In [37]:
def flatten(value: dict | list | str | int | float | bool | None, result: dict, base_json_path: str = "$") -> dict:
    r"""Translates the specified value into a Python dictionary, which can be written to a CSV string.

    Args:
        value: The value you want to translate, originally read from a valid JSON string.
        result: The Python dictionary to which you want to add the translated value.
        base_json_path: The JSONPath expression that indicates where the value was read from.

    Returns:
        A Python dictionary that contains the translated value, ready to be written to a CSV string via
        an instance of `csv.DictWriter` (see: https://docs.python.org/3/library/csv.html#csv.DictWriter).

    Note: This function invokes itself recursively.
    
    In the resulting dictionary:
    - Each key is a JSONPath expression indicating where the primitive value originated within the JSON value.
    - Each value is the primitive value at that location within the JSON value.

    For example:
    - The JSONPath expression, `$.has_input[1].name`, refers to the `name` property of the object
      that is the second element (0-indexed) of the array in the `has_input` property of the root value.
    
    # Primitive values:
    >>> flatten(1, {})  # int
    {'$': 1}
    >>> flatten(0.1, {})  # float
    {'$': 0.1}
    >>> flatten("potato", {})  # str
    {'$': 'potato'}
    >>> flatten(True, {})  # bool
    {'$': True}
    >>> flatten(None, {})  # None
    {'$': None}
    
    # Invalid value:
    >>> flatten(lambda: 123, {})  # function
    Traceback (most recent call last):
    ...
    ValueError: The data type of the value is invalid.

    # Empty dict and list values:
    >>> flatten({}, {})  # empty dict
    {}
    >>> flatten([], {})  # empty list
    {}

    # Non-empty dict and list values:
    >>> flatten({"a": 1}, {})  # dict having 1 key-value pair
    {'$.a': 1}
    >>> flatten([1], {})  # list having 1 element
    {'$[0]': 1}

    # Compound value:
    >>> flatten([{"a": 1}, {"a": 2, "b": 3}], {})  # list of dicts
    {'$[0].a': 1, '$[1].a': 2, '$[1].b': 3}
    >>> flatten({"a": [1], "b": [2, 3]}, {})  # dict of lists
    {'$.a[0]': 1, '$.b[0]': 2, '$.b[1]': 3}

    # (Mostly) real-world API response value:
    #
    # Note: In this doctest, we `pprint` the resulting dictionary so it spans multiple lines.
    #       We find that easier to read compared to all key-value pairs being on a single line.
    #
    >>> from pprint import pprint
    >>> api_response_body = {
    ...     "resources": [
    ...         {
    ...         "id": "nmdc:wfmag-11-00jn7876.2",
    ...         "name": "My workflow execution",
    ...         "started_at_time": "2024-03-24T16:04:04.936972+00:00",
    ...         "ended_at_time": "2024-03-24T17:49:34.756540+00:00",
    ...         "was_informed_by": [
    ...             "nmdc:omprc-11-7yj0jg57"
    ...         ],
    ...         "execution_resource": "NERSC-Perlmutter",
    ...         "git_url": "https://github.com/microbiomedata/metaMAGs",
    ...         "has_input": [
    ...             "nmdc:dobj-11-yjp1xw52",
    ...             "nmdc:dobj-11-3av14y79",
    ...             "nmdc:dobj-11-wa5pnq42"
    ...         ],
    ...         "type": "nmdc:MagsAnalysis",
    ...         "has_output": [],
    ...         "version": "v1.1.0",
    ...         "processing_institution": "NMDC"
    ...         }
    ...     ],
    ...     "next_page_token": "nmdc:sys0xvg3j376"
    ... }
    >>> result = flatten(api_response_body, {})
    >>> pprint(result)
    {'$.next_page_token': 'nmdc:sys0xvg3j376',
     '$.resources[0].ended_at_time': '2024-03-24T17:49:34.756540+00:00',
     '$.resources[0].execution_resource': 'NERSC-Perlmutter',
     '$.resources[0].git_url': 'https://github.com/microbiomedata/metaMAGs',
     '$.resources[0].has_input[0]': 'nmdc:dobj-11-yjp1xw52',
     '$.resources[0].has_input[1]': 'nmdc:dobj-11-3av14y79',
     '$.resources[0].has_input[2]': 'nmdc:dobj-11-wa5pnq42',
     '$.resources[0].id': 'nmdc:wfmag-11-00jn7876.2',
     '$.resources[0].name': 'My workflow execution',
     '$.resources[0].processing_institution': 'NMDC',
     '$.resources[0].started_at_time': '2024-03-24T16:04:04.936972+00:00',
     '$.resources[0].type': 'nmdc:MagsAnalysis',
     '$.resources[0].version': 'v1.1.0',
     '$.resources[0].was_informed_by[0]': 'nmdc:omprc-11-7yj0jg57'}
    """

    # If the value is a primitive, store it as is.
    if isinstance(value, (str, int, float, bool)) or value is None:
        result.update({base_json_path: value})
    # Else, if the value is a dictionary, process each of its key-value pairs.
    elif isinstance(value, dict):
        for key, val in value.items():
            result.update(flatten(val, result, f"{base_json_path}.{key}"))
    # Else, if the value is a list, process each of its elements.
    elif isinstance(value, list):
        for index, val in enumerate(value):
            result.update(flatten(val, result, f"{base_json_path}[{index}]"))
    # Else, raise an exception indicating the value is invalid.
    else:
        raise ValueError("The data type of the value is invalid.")

    return result

Here's how you can use it.

In [ ]:
!python -m pip install httpx

Send a request to the NMDC Runtime API.

The body of the response from the API will be a JSON string.

> Note: Normally, we'd access the response body using `response.json()`, which automatically parses it into a Python value (e.g. a `dict`). In this case, since we specifically want to demonstrate _JSON_-to-CSV conversion, we access the raw JSON response via `response.text`.

In [39]:
import httpx  # docs: https://www.python-httpx.org/


# Get a study from the NMDC Runtime API.
response = httpx.get("https://api.microbiomedata.org/nmdcschema/study_set?max_page_size=1")
response_body: str = response.text
response_body

'{"resources":[{"id":"nmdc:sty-11-076c9980","name":"Lab enrichment of tropical soil microbial communities from Luquillo Experimental Forest, Puerto Rico","description":"This study is part of the Microbes Persist: Systems Biology of the Soil Microbiome Science Focus Area, which seeks to understand how microbial ecophysiology, population dynamics, and microbe–mineral–organic matter interactions regulate the persistence of microbial residues in soil under changing moisture regimes.","ecosystem":"Environmental","ecosystem_category":"Terrestrial","ecosystem_type":"Soil","ecosystem_subtype":"Unclassified","specific_ecosystem":"Forest Soil","principal_investigator":{"has_raw_value":"Jennifer Pett-Ridge","profile_image_url":"https://portal.nersc.gov/project/m3408/profile_images/pett-ridge_jennifer.jpg","orcid":"orcid:0000-0002-4439-2398","type":"nmdc:PersonValue"},"type":"nmdc:Study","websites":["https://sc-programs.llnl.gov/biological-and-environmental-research-at-llnl/soil-microbiome"],"titl

Now that we have a JSON string, let's translate it into a CSV string.

In [41]:
from csv import DictWriter
from io import StringIO
import json


def translate_json_into_csv(json_string: str) -> str:
    """Translates the specified JSON string into a CSV string."""
    
    # Parse the JSON string into a Python value.
    value = json.loads(json_string)

    # Flatten the Python value into a dictionary that can be written to a CSV string.
    flattened_value = flatten(value, {})

    # Write the flattened dictionary to a CSV string.
    csv_string_io = StringIO()
    csv_writer = DictWriter(csv_string_io, fieldnames=flattened_value.keys())
    csv_writer.writeheader()
    csv_writer.writerow(flattened_value)
    csv_string = csv_string_io.getvalue()
    csv_string_io.close()

    return csv_string


csv_string: str = translate_json_into_csv(response_body)
print(csv_string)

$.resources[0].id,$.resources[0].name,$.resources[0].description,$.resources[0].ecosystem,$.resources[0].ecosystem_category,$.resources[0].ecosystem_type,$.resources[0].ecosystem_subtype,$.resources[0].specific_ecosystem,$.resources[0].principal_investigator.has_raw_value,$.resources[0].principal_investigator.profile_image_url,$.resources[0].principal_investigator.orcid,$.resources[0].principal_investigator.type,$.resources[0].type,$.resources[0].websites[0],$.resources[0].title,$.resources[0].funding_sources[0],$.resources[0].funding_sources[1],$.resources[0].has_credit_associations[0].applied_roles[0],$.resources[0].has_credit_associations[0].applied_roles[1],$.resources[0].has_credit_associations[0].applied_roles[2],$.resources[0].has_credit_associations[0].applied_roles[3],$.resources[0].has_credit_associations[0].applied_roles[4],$.resources[0].has_credit_associations[0].applied_roles[5],$.resources[0].has_credit_associations[0].applied_roles[6],$.resources[0].has_credit_associati

Here's what that same CSV string looks like as a table.

In [ ]:
!python -m pip install pandas

In [43]:
import pandas as pd


dataframe = pd.read_csv(StringIO(csv_string))
dataframe

,$.resources[0].id,$.resources[0].name,$.resources[0].description,$.resources[0].ecosystem,$.resources[0].ecosystem_category,$.resources[0].ecosystem_type,$.resources[0].ecosystem_subtype,$.resources[0].specific_ecosystem,$.resources[0].principal_investigator.has_raw_value,$.resources[0].principal_investigator.profile_image_url,...,$.resources[0].associated_dois[1].type,$.resources[0].associated_dois[2].doi_value,$.resources[0].associated_dois[2].doi_category,$.resources[0].associated_dois[2].doi_provider,$.resources[0].associated_dois[2].type,$.resources[0].associated_dois[3].doi_value,$.resources[0].associated_dois[3].doi_provider,$.resources[0].associated_dois[3].doi_category,$.resources[0].associated_dois[3].type,$.next_page_token
0,nmdc:sty-11-076c9980,Lab enrichment of tropical soil microbial comm...,This study is part of the Microbes Persist: Sy...,Environmental,Terrestrial,Soil,Unclassified,Forest Soil,Jennifer Pett-Ridge,https://portal.nersc.gov/project/m3408/profile...,...,nmdc:Doi,doi:10.46936/sthm.proj.2017.49804/60006181,award_doi,emsl,nmdc:Doi,doi:10.25345/C58C9RF3K,massive,dataset_doi,nmdc:Doi,nmdc:sys0x13bzq51


Yahoo! That's a CSV-formatted version of the API response.